<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica4%3A%20Filtrado%20en%20frecuencia/python/Pr%C3%A1ctica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/atico.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/bars.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/rocas.jpg"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/lena.png"

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

 # Parte 1

In [ ]:
I = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
F = np.fft.fft2(I)
IShift = np.fft.fftshift(I)
Fshift = np.fft.fftshift(F)

La siguiente línea dará un error

In [ ]:
plt.imshow(F, cmap="gray")

Debido al tipo de dato que queremos desplegar

In [ ]:
print(F)

In [ ]:
plt.imshow(np.abs(F), cmap="gray")

La línea anterior sigue sin mostrar los resultados correctamente debido a la escala a la que se encuentran los datos

In [ ]:
plt.imshow(np.log1p(np.abs(F)), cmap='gray')

Mostrar resultados

In [ ]:
#fig, ax = plt.subplots(2,2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(2,2,1)
plt.imshow(I, cmap="gray")
plt.title("Original")
plt.subplot(2,2,2)
plt.imshow(np.log1p(np.abs(F)), cmap='gray')
plt.title("Frecuencia")
plt.subplot(2,2,3)
plt.imshow(IShift, cmap="gray")
plt.subplot(2,2,4)
plt.imshow(np.log1p(np.abs(Fshift)), cmap='gray')
plt.title("FFT-Shift")

## Filtrado

## Filtros Ideales

Filtro

In [ ]:
def filtro(D0, filter):
  M, N = F.shape
  H = np.zeros((M, N), dtype=np.float32)
  for u in range(M):
      for v in range(N):
          D = np.sqrt((u - M/2)**2 + (v - N/2)**2)
          if filter == "lpf":
          # ----Pasa bajas ideal-----
            if D <= D0:
                H[u, v] = 1
            else:
                H[u, v] = 0
          # ----Pasa altas ideal----
          elif filter == "hpf":
            if D <= D0:
                H[u, v] = 0
            else:
                H[u, v] = 1
          else:
            print("Error en selección del filtro")
  return H

In [ ]:
H = filtro(30, "lpf")
plt.imshow(H, cmap="binary")
plt.axis("off")

Filtrado

In [ ]:
Gshift = Fshift * H #Filtradoq
G = np.fft.fftshift(Gshift)
g = np.abs(np.fft.ifft2(G))

fig, ax = plt.subplots(1, 3, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,3,1)
plt.imshow(np.log1p(np.abs(Gshift)), cmap='gray')
plt.title("G shift")
plt.subplot(1,3,2)
plt.imshow(np.log1p(np.abs(G)), cmap='gray')
plt.title("G")
plt.subplot(1,3,3)
plt.imshow(g, cmap='gray')
plt.title("g")

Comparativa con diferentes $D0$

In [ ]:
filter = "lpf"

fa, fb, fc = 10, 70, 280


GA = np.fft.fftshift(Fshift * filtro(fa, filter))
GB = np.fft.fftshift(Fshift * filtro(fb, filter))
GC = np.fft.fftshift(Fshift * filtro(fc, filter))

fig = plt.subplots(2, 3, figsize=(20,10))
plt.subplot(2,3,1)
plt.imshow(filtro(fa, filter), cmap='binary')
plt.subplot(2,3,2)
plt.imshow(filtro(fb, filter), cmap='binary')
plt.subplot(2,3,3)
plt.imshow(filtro(fc, filter), cmap='binary')
plt.subplot(2,3,4)
plt.imshow(np.abs(np.fft.ifft2(GA)), cmap='gray')
plt.subplot(2,3,5)
plt.imshow(np.abs(np.fft.ifft2(GB)), cmap='gray')
plt.subplot(2,3,6)
plt.imshow(np.abs(np.fft.ifft2(GC)), cmap='gray')

## Filtro box

Leer imagen

In [ ]:
I = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
F = np.fft.fft2(I)
Fshift = np.fft.fftshift(F)

fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(I, cmap="gray")
plt.title("Original")
plt.subplot(1,2,2)
plt.imshow(np.log1p(np.abs(F)), cmap='gray')
plt.title("Frecuencia")

Filtro box

In [ ]:
M, N = F.shape
H = np.zeros((M, N), dtype=np.float32)
n = 11
H[0:n, 0:n] = 1/(n*n)

fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(np.real(H), cmap='gray')
plt.title("h")
plt.subplot(1,2,2)

H = np.fft.fft2(H)
plt.imshow(np.abs(np.fft.fftshift(H)), cmap='gray')
plt.title("H")

Filtrado

In [ ]:
G = F * H
%timeit G = F * H

In [ ]:
n = 11
h = np.ones((n,n)) / (n*n)
%timeit G2 = cv2.filter2D(I, -1, h)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
fig.tight_layout()
plt.clf()

plt.subplot(1,2,1)
plt.imshow(np.log1p(np.abs(np.fft.fftshift(G))), cmap='gray')
plt.title("G")

plt.subplot(1,2,2)
g = np.abs(np.fft.ifft2(G))
plt.imshow(g, cmap='gray')
plt.title("g")

# Parte 2: Importancia de la fase

In [ ]:
I2 = cv2.resize(cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE), (300,300))
I1 = cv2.resize(cv2.imread('atico.jpg', cv2.IMREAD_GRAYSCALE), (300,300))

plt.subplot(121)
plt.imshow(I1, cmap='gray')
plt.subplot(122)
plt.imshow(I2, cmap='gray')
plt.show()

In [42]:
I1_fft = np.fft.fftshift(np.fft.fft2(I1))
I2_fft = np.fft.fftshift(np.fft.fft2(I2))

I1_amplitude = np.sqrt(np.real(I1_fft) ** 2 + np.imag(I1_fft) ** 2)
I1_phase = np.arctan2(np.imag(I1_fft), np.real(I1_fft))
I2_amplitude = np.sqrt(np.real(I2_fft) ** 2 + np.imag(I2_fft) ** 2)
I2_phase = np.arctan2(np.imag(I2_fft), np.real(I2_fft))

Mostrar únicamente la amplitud (hacer que la fase sea cero).

In [ ]:
plt.subplot(221)
plt.imshow(I1, cmap='gray')

plt.subplot(222)
I1A = np.real(np.fft.ifft2(I1_amplitude))
# Mapear de 0 a 255
I1A[I1A > 255] = 255
I1A[I1A < 0] = 0
plt.imshow(I1A, cmap="gray")

plt.subplot(223)
plt.imshow(I2, cmap='gray')

plt.subplot(224)
I2A = np.real(np.fft.ifft2(I2_amplitude))
# Mapear de 0 a 255
I2A[I2A > 255] = 255
I2A[I2A < 0] = 0
plt.imshow(I2A, cmap="gray")


No podríamos hacer lo mismo con la fase, pues si hacemos 0 la amplitud se perdería la imagen por completo

In [ ]:
plt.subplot(221)
plt.imshow(I1, cmap='gray')

plt.subplot(222)
I1P = np.real((np.fft.ifft2(I1_phase)))
plt.imshow(I1P.astype('uint8'), cmap="gray")

plt.subplot(223)
plt.imshow(I2, cmap='gray')

plt.subplot(224)
I2P = np.real((np.fft.ifft2(I2_phase)))
plt.imshow(I2P.astype('uint8'), cmap="gray")

Como no podemos hacer la magnitud igual a cero, lo que haremos ahora será mezclar la amplitud de una con la fase de la otra.

In [43]:
I1_I2_comb = np.multiply(I1_amplitude, np.exp(1j * I2_phase))
I1_I2 = np.real(np.fft.ifft2(I1_I2_comb))

Mostrar resultados y hacer un mapeo para valores negativos a cero y 255 como máximo

In [ ]:
plt.figure(figsize=(15, 20))
plt.subplot(131)
plt.imshow(np.abs(I1_I2), cmap='gray')
# plt.subplot(132)
# I1_I2_shift = I1_I2 + I1_I2.min()
# I1_I2_shift[I1_I2_shift>255] = 255
# plt.imshow(I1_I2_shift, cmap="gray")
# plt.subplot(133)
# I1_I2[I1_I2>255] = 255
# I1_I2[I1_I2 <0] = 0
# plt.imshow(I1_I2, cmap="gray")

La amplitud es solo la intensidad sin tener en cuenta la posición de esa intensidad, es decir, no contiene información espacial. La fase contiene información de como esas intensidades recaen sobre otras.

# Parte 3: Imágenes híbridas

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/marilyn.png"
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/einstein.jpg"

Filtro gaussiano

In [ ]:
def gaussian(r2, std=1):
    return np.exp(-r2/(2.*std**2)) / (2.*np.pi*std**2)
def make_gaussian(std=1, size=None):
    if size is None:
        size = np.floor(6 * std)
        if size % 2 == 0:
            size = size - 1
        else:
            size= size - 2
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    distance = x**2+ y**2
    kernel = gaussian(r2=distance, std=std)
    return kernel / kernel.sum()

Imágenes de entrada

In [ ]:
I1 = cv2.resize(cv2.imread('marilyn.png', cv2.IMREAD_GRAYSCALE), (300,300))
I2 = cv2.resize(cv2.imread('einstein.jpg', cv2.IMREAD_GRAYSCALE), (300,300))

plt.subplot(121)
plt.imshow(I1, cmap='gray')
plt.subplot(122)
plt.imshow(I2, cmap='gray')
plt.show()

Pasar al dominio de la frecuencia

In [ ]:
I1_fft = np.fft.fftshift(np.fft.fft2(I1))
I2_fft = np.fft.fftshift(np.fft.fft2(I2))

Filtros pasa bajas y pasa altas

In [ ]:
cutoff_lpf = 10
cutoff_hpf = 20

lpf = make_gaussian(cutoff_lpf)
hpf = lpf.max() - lpf

# padding para hacer que tengan el mismo tamaño que las imágenes de entrada
lpf = np.pad(lpf, [(I1.shape[0]//2-lpf.shape[0]//2,
                  I1.shape[0]//2-lpf.shape[0]//2 - 1),
                 (I1.shape[1]//2-lpf.shape[1]//2,
                  I1.shape[1]//2-lpf.shape[1]//2 - 1)], mode='constant', constant_values=0)

hpf = np.pad(hpf, [(I1.shape[0]//2-hpf.shape[0]//2,
                  I1.shape[0]//2-hpf.shape[0]//2 - 1),
                 (I1.shape[1]//2-hpf.shape[1]//2,
                  I1.shape[1]//2-hpf.shape[1]//2 - 1)], mode='constant', constant_values=lpf.max())

plt.subplot(121)
plt.imshow(lpf, cmap='gray')
plt.subplot(122)
plt.imshow(hpf, cmap='gray')

Filtramos las imágenes y aplicamos la transformada inversa para regresarlas al espacio de la imagen.

In [ ]:
combined = np.fft.ifft2(np.multiply(I1_fft, lpf) + np.multiply(I2_fft, hpf))

Mostrar el resultado.

¿Cambia cuando se ve a diferentes distancias?

Cuando hacemos un submuestreo (nos alejamos), perdemos altas frecuencias. Cuando nos acercamos podemos ver más detalles (recuperamos altas frecuencias).

Estamos filtrando a Marilyn con el pasabajas y Einstein con el pasaaltas. Así, al alejarnos estamos perdiendo la imagen de alta frecuencia (Einstein) y al acercarnos predominan.

In [ ]:
plt.imshow(np.abs(combined), cmap='gray')

# Parte 4: Detección de imágenes borrosas


In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica4%3A%20Filtrado%20en%20frecuencia/data/blurry.jpg"

In [ ]:
gray = cv2.imread("blurry.jpg", cv2.IMREAD_GRAYSCALE)

fft = np.fft.fft2(gray)
fftShift = np.fft.fftshift(fft)

Calcular la magnitud de la imagen y luego la media

In [ ]:
magnitude = 20 * np.log(np.abs(fft))
mean = np.mean(magnitude)

la imagen se considerará borrosa si la media de su magnitud está por debajo del umbral.

In [ ]:
umbral = 130
print("Media: ", mean)
if mean <= umbral:
  print("Borrosa")
else:
  print("Nítida")

plt.imshow(gray, cmap="gray")